In [1]:
import numpy as np
import math
import pprint as pp
from scipy import *
from scipy.sparse import *

## Generation de graphes

In [88]:
handmadeGraph1 = {1: {2:0.5,3:0.5},
                 2: {4:0.6},
                 3: {4:0.4,5:0.6},
                 4: {},
                 5: {},
                }

handmadeGraph=dok_matrix((7,7),dtype=float32)
handmadeGraph[0,1] =0.5
handmadeGraph[0,2] =0.5
handmadeGraph[1,3] =0.6
handmadeGraph[2,3] =0.4
handmadeGraph[2,4] =0.6

print("Handmade Graph")
print(handmadeGraph)


nodes = np.arange(handmadeGraph.shape[0])

def randomGraph(nodes):
    m=len(nodes)
    graph = dok_matrix((m,m),dtype=float32)
    for i in range(m):
        for j in range(m):
            if(i!=j):
                graph[i,j]=np.random.rand()
    return graph
print("Random Graph")
print(randomGraph(nodes))

Handmade Graph
  (0, 1)	0.5
  (0, 2)	0.5
  (1, 3)	0.6
  (2, 3)	0.4
  (2, 4)	0.6
Random Graph
  (0, 1)	0.85509074
  (0, 2)	0.720146
  (0, 3)	0.30122247
  (0, 4)	0.65089786
  (0, 5)	0.14567702
  (0, 6)	0.35609236
  (1, 0)	0.31820923
  (1, 2)	0.90772665
  (1, 3)	0.1180029
  (1, 4)	0.34742156
  (1, 5)	0.9842773
  (1, 6)	0.16153888
  (2, 0)	0.77540857
  (2, 1)	0.416217
  (2, 3)	0.7757969
  (2, 4)	0.7786772
  (2, 5)	0.36143216
  (2, 6)	0.0063396688
  (3, 0)	0.31420955
  (3, 1)	0.46347323
  (3, 2)	0.30967507
  (3, 4)	0.7745039
  (3, 5)	0.99524075
  (3, 6)	0.7713674
  (4, 0)	0.10656903
  (4, 1)	0.7530032
  (4, 2)	0.6491206
  (4, 3)	0.56293464
  (4, 5)	0.04105433
  (4, 6)	0.96783847
  (5, 0)	0.26462764
  (5, 1)	0.3428392
  (5, 2)	0.32807562
  (5, 3)	0.59692323
  (5, 4)	0.51387095
  (5, 6)	0.34040385
  (6, 0)	0.95887685
  (6, 1)	0.049239907
  (6, 2)	0.47832248
  (6, 3)	0.0840934
  (6, 4)	0.61603975
  (6, 5)	0.8021358


## Inference 
Generation de cascades

In [89]:

def genCascade(graph,startNode,startTime=0):
    ''' Receive graph and starting infected node generate a cascade 
        Args
            graph(node to child representation)
            cascade {node : time} 
    '''
    cascade = {startNode : startTime}
    lastInfected = [startNode]
    infected_next = {}
    time = startTime+1
    while len(lastInfected)> 0:
        for infected in lastInfected:
            for (_,child),pct in graph[infected,:].items():
                if (child not in cascade) and (child not in infected_next):
                    if (np.random.rand()<pct):
                        infected_next[child] = time
        cascade.update(infected_next)
        lastInfected = list(infected_next.keys())
        infected_next = {}
        time +=1
        
    return cascade

In [90]:
nbCascades = 200
cascades = [genCascade(handmadeGraph,0) for i in range(nbCascades)]
pp.pprint(cascades[:3])

[{0: 0, 2: 1, 3: 2}, {0: 0, 2: 1, 4: 2}, {0: 0, 2: 1, 4: 2}]


## Independant cascades apprentissage


### 2008 Saito
Les temps d'infections contiguë <br>
$D(i)$  all newly infected at time $i$  <br>
$D = D(0) \cup D(1) \cup ... \cup D(n)$
#### Maximimum likelyhood expectation
Etapes : 
1. On donne une première estimation des arêtes du graph : $k_{u,v}$ <br>
- On calcule $P_{w}$ en fonction de $k_{u,v}$
- On calcule par ML $k_{u,v}$ en fonction de $P_{w}$
- Retour à l'étape 2 jusqu'à convergence


In [91]:
def cascade_repr(cascade):
    ''' Transforme une cascade en list ayant pour indice 
        le temps d'infection t et pour contenu une list contenant les noeuds infectés au temps t'''
    
    maxT  = max(cascade.values())+1
    Ds = [[] for i in range(maxT)]
    for (n,t) in cascade.items():
        Ds[t].append(n)
    return Ds
D = [cascade_repr(c) for c in cascades]
print(cascades[:2],'\n',D[:2])

[{0: 0, 2: 1, 3: 2}, {0: 0, 2: 1, 4: 2}] 
 [[[0], [2], [3]], [[0], [2], [4]]]


In [92]:
def nodes_in_Ds(Ds):
    uniques = []
    for nodes in Ds : 
        for n in nodes : 
            uniques.append(n)
    return uniques

In [94]:

def P_sw(g,Ds,w):
    ''' Vraisemblance de l infection d un noeud sachant le graphe 
    et les noeuds acivés à l'etape precedente '''
    not_activated = 1
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        raise Exception(f"node {w} is not in diff episode : {Ds} ")
    else :  # si le noeud est dans l'épisode de diffusion
        return 1 - np.prod ([1-g[parent,w] for parent in Ds[t-1]])

In [95]:
def Expectation(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [96]:
def D_plus_uv_id(D,u,v):
    ''' pour chaque couples u,v renvoit l'ensemble des 
    Ds (episode infections) ou u precede v'''
    D_plus = []
    for i,Ds in enumerate(D): 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v in Ds[t]):
                D_plus.append(i)
                break
    return D_plus

def D_minus_uv_len(D,u,v):
    '''Pour chaque couple u,v renvoit le cardinal de l'ensemble des 
    Ds(episode infection) ou u est present mais v ne le suit pas'''
    D_minus_len=0
    for Ds in D : 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v not in Ds[t]):
                D_minus_len+=1
                break
        if (u in Ds[-1]):
            D_minus_len+=1
    return D_minus_len

In [98]:
def Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v):
    '''Calcule les nouveaux paramètre pour l'arete u,v '''
    if ((len(D_plus_id[u][v])+Dminus_len[u][v]) == 0):
        raise Exception(f"{u}-{v} Division zero")
    return (1/(len(D_plus_id[u][v])+Dminus_len[u][v])) *sum([g[u,v]/p_sw[i][v] for i in D_plus_id[u][v]])


def Maximisation(g,D_plus_id,Dminus_len,p_sw):
    ''' Calcule les nouveaux paramètres pour le graphe'''
    gprime = dok_matrix(g.shape,dtype=float32)
    for u,v in g.keys():
        if u != v:
            gprime[u,v] = Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v)
    return gprime


In [99]:
def EM_IC(D,nodes):
        # initalisation
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv_len(D,v,u)for u in nodes} for v in nodes}
    for i in range(100):
        p_sw = Expectation(g,D)
        g = Maximisation(g,D_plus_id,D_minus_len,p_sw)
    return g

nodes = set()
for c in cascades:
    nodes.update(c.keys())
print(nodes)

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC(D,nodes)
print(finalGraph)

{0, 1, 2, 3, 4}


/home/yass/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  """


  (0, 1)	0.495
  (0, 2)	0.56
  (1, 3)	0.599039
  (1, 4)	2.1120464e-31
  (2, 3)	0.39005125
  (2, 4)	0.54464287


### Methode 2
Les temps d'infections non contiguë : 
Un noeud peut désorais être infecté non seulement par ceux qui ont été infecté

In [100]:
def P_sw2(g,Ds,w):
    ''' Vraisemblance de l infection d un noeud sachant le graphe 
    et les tous noeuds précedemment acivés '''
    not_activated = 1
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        raise Exception(f"node {w} is not in diff episode : {Ds} ")
    else :  # si le noeud est dans l'épisode de diffusion
        preceding_nodes = [] # on regroupe les noeuds des épisodes précedents
        for nodes in Ds[:t]:
            for n in nodes : 
                preceding_nodes.append(n)
        
        return 1 - np.prod ([1-g[parent,w] for parent in preceding_nodes])

In [101]:
def Expectation2(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw2(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [102]:
def D_plus_uv_id2(D,u,v):
    ''' Pour chaque couples u,v renvoit l'ensemble des 
        Ds (episode infections) ou t_u < t_v'''
    D_plus = []
    
    for i,Ds in enumerate(D) : 
        preceding_nodes = []
        for t in range(0,len(Ds)):
            if (v in Ds[t]):
                if (u in preceding_nodes):
                    D_plus.append(i)
                    break
            else : 
                preceding_nodes +=Ds[t]
    return D_plus

def D_minus_uv2_len(D,u,v):
    '''Pour chaque couple u,v renvoit l'ensemble des 
    Ds(episode infection) ou u est present et non(t_u < t_v)'''
    D_minus_len = 0
    for Ds in D : 
        u_in_Ds = False
        v_in_Ds = False
        for t in range(0,len(Ds)):
            if (u in Ds[t]):
                u_in_Ds = True
            if (u_in_Ds and v in Ds[t]):
                v_in_Ds = True
                break
        if (u_in_Ds and not v_in_Ds):
            D_minus_len+=1
    return D_minus_len

In [103]:
def EM_IC_2(D,nodes):
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id2(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv2_len(D,v,u)for u in nodes} for v in nodes}
    
    for i in range(100):
        p_sw = Expectation2(g,D)
        g = Maximisation(g,D_plus_id,D_minus_len,p_sw)
    return g

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC_2(D,nodes)
print(finalGraph)

/home/yass/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  """


  (0, 1)	0.495
  (0, 2)	0.56
  (0, 3)	6.0593626e-12
  (0, 4)	1.6637431e-26
  (1, 3)	0.599039
  (1, 4)	4.449825e-31
  (2, 3)	0.39005125
  (2, 4)	0.54464287


## Evaluation